In [1]:
import pandas as pd
import dbutil
import KlineService
from datetime import datetime, date
from BollBand import DayBollBand,WeekBollBand,MonthBollBand,SixtyMinBollBand,FiveMinBollBand
from IPython.core.debugger import set_trace
import talib

##
#  选择不同的BollBand Collection
#
def chooseBollBandCollection(period):
    periodBollBandCollection = {
        "day" : "BollBand_Day",
        "week" : "BollBand_Week",
        "month" : "BollBand_Month",
        "5m" : "BollBand_5m",
        "15m" : "BollBand_15m",
        "30m" : "BollBand_30m",
        "60m" : "BollBand_60m"
    }
    return periodBollBandCollection.get(period)

##
#  写BollBand数据库
#
#
def writeBollBandToDb(period, stockCode, stockName, boll_df):
    dataList = []
    for index,bollBand in boll_df.iterrows():
        bollBandDate = bollBand['date']
        if period == "day":
            bollBandObj = DayBollBand(stockCode, stockName)
        elif period == "week":
            bollBandObj = WeekBollBand(stockCode, stockName)
        elif period == "month":
            bollBandObj = MonthBollBand(stockCode, stockName)
        elif period == "5m":
            bollBandObj = FiveMinBollBand(stockCode, stockName)
        elif period == "15m":
            bollBandObj = FiftyMinBollBand(stockCode, stockName)
        elif period == "30m":
            bollBandObj = ThirtyMinBollBand(stockCode, stockName)
        elif period == "60m":
            bollBandObj = SixtyMinBollBand(stockCode, stockName)

        bollBandObj.date = bollBandDate
        bollBandObj.upper = bollBand['upper']
        bollBandObj.middle = bollBand['middle']
        bollBandObj.lower = bollBand['lower']
        bollBandObj.closePrice = bollBand['closePrice']
        bollBandObj.overBuy = bollBand['overBuyFlag']
        bollBandObj.overSell = bollBand['overSellFlag']
        
        dataList.append(bollBandObj.__dict__)
        
    mydb = dbutil.connectDB()
    collection = mydb[chooseBollBandCollection(period)]
    if len(dataList) > 0:
        collection.insert_many(dataList)
    else:
        raise RuntimeError("BollBand数据为空")
        
def computeStockBollBand(period, stockCode, stockName, startDate, endDate):
    try:
#       compute1 = datetime.datetime.now().timestamp()
        df = KlineService.readStockKline(stockCode, period, startDate, endDate)
#       compute2 = datetime.datetime.now().timestamp()
#       print("read stockLine:", compute2 - compute1)
        if period == "day":
            # 剔除日线停盘数据
            df = df[df['tradeStatus'] == '1']
        closePrice = df["closePrice"]
        upper, middle, lower = talib.BBANDS(closePrice, timeperiod=22, nbdevup=2, nbdevdn=2, matype=0)
        set_trace()
        data = {
            "upper":upper,
            "middle":middle,
            "lower":lower,
            "closePrice":closePrice
        }
        bollDF = pd.DataFrame(data)
        bollDF["date"] = df["date"]
        bollDF["overBuyFlag"] = False
        bollDF["overSellFlag"] = False
#       compute3 = datetime.datetime.now().timestamp()
#       print("compute rsi:", compute3 - compute2)
        writeBollBandToDb(period, stockCode, stockName, bollDF)
#       compute4 = datetime.datetime.now().timestamp()
#       print("write to db:", compute4 - compute3)
        return True
    except BaseException as e:
        print ("download " + stockCode + " error:" + str(e))
        return False

In [2]:
computeStockBollBand("60m", "sz.002030", "达安基因", "2017-01-01", "2019-09-27")

> <ipython-input-1-63fabcdf4197>(77)computeStockBollBand()
     75         set_trace()
     76         data = {
---> 77             "upper":upper,
     78             "middle":middle,
     79             "lower":lower,

ipdb> c


True